In [1]:
from scipy.spatial.distance import euclidean
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2

from matplotlib import pyplot as plt

In [2]:
# Function to show array of images (intermediate results)
def show_images(images):
	for i, img in enumerate(images):
		cv2.imshow("image_" + str(i), img)
	cv2.waitKey(0)
	cv2.destroyAllWindows()
    
def padding_by_zero(frame, height, width):
    # 依照長邊縮放，並保持比例
    if height > width:
        new_height = 640
        new_width = int(width * (640 / height))
    else:
        new_width = 640
        new_height = int(height * (640 / width))
    
    # 缩放影像
    frame_resized = cv2.resize(frame, (new_width, new_height))

    # 創建黑色背景並將縮放後的影像貼到中心
    top = (640 - new_height) // 2
    bottom = 640 - new_height - top
    left = (640 - new_width) // 2
    right = 640 - new_width - left
    
    # 將影像填充到 640x640，並保持比例
    pad = cv2.copyMakeBorder(frame_resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    #print(new_width, new_height)
    
    return pad

In [3]:
def contour_size(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    edged = cv2.Canny(blur, 50, 100)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    #最左上
    cnts = sorted(cnts, key=lambda c: cv2.boundingRect(c)[0] + cv2.boundingRect(c)[1])
    cnts = [x for x in cnts if 3000 < cv2.contourArea(x) < 5000]

    cv2.drawContours(frame, cnts, -1, (0,255,0), 3)
    
    if len(cnts) == 0:
        #print("沒有找到足夠大的輪廓")
        return image  # 或者根據需求處理此情況

    print(cv2.contourArea(cnts[0]))
    ref_object = cnts[0]
    box = cv2.minAreaRect(ref_object)
    box = cv2.boxPoints(box)
    box = np.array(box, dtype="int")
    box = perspective.order_points(box)
    (tl, tr, br, bl) = box

    dist_in_pixel = euclidean(tl, tr)
    dist_in_cm = 2
    pixel_per_cm = dist_in_pixel / dist_in_cm
    
    for cnt in cnts:
        box = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        box = perspective.order_points(box)
        (tl, tr, br, bl) = box
        cv2.drawContours(image, [box.astype("int")], -1, (0, 0, 255), 2)
        mid_pt_horizontal = (tl[0] + int(abs(tr[0] - tl[0]) / 2), tl[1] + int(abs(tr[1] - tl[1]) / 2))
        mid_pt_vertical = (tr[0] + int(abs(tr[0] - br[0]) / 2), tr[1] + int(abs(tr[1] - br[1]) / 2))
        wid = euclidean(tl, tr) / pixel_per_cm
        ht = euclidean(tr, br) / pixel_per_cm
        cv2.putText(image, "{:.1f}cm".format(wid), (int(mid_pt_horizontal[0] - 15), int(mid_pt_horizontal[1] - 10)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        cv2.putText(image, "{:.1f}cm".format(ht), (int(mid_pt_vertical[0] + 10), int(mid_pt_vertical[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
    
    return image

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 60)

while True:
    ret, frame = cap.read()
    height, width, channels = frame.shape
    frame_padded = padding_by_zero(frame, height, width)

    out = contour_size(frame_padded)
    
    cv2.imshow("Webcam", out) # This will open an independent window
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cap.release()
        break
        
cv2.destroyAllWindows() 
cv2.waitKey(1)

3656.0
3618.5
3584.0
3703.5
3996.5
4067.0
3913.5
3653.0
3642.0
3559.5
3616.5
3614.0
3623.0
3615.0
3617.0
3614.5
3617.0
3620.5
3617.0
3624.0
3618.0
3617.5
3614.5
3614.5
3622.0
3615.0
3621.0
3613.0
3614.0
3621.0
3612.0
3615.5
3621.5
3612.5
3613.0
3617.0
3616.0
3617.0
3621.5
3615.0
3613.5
3620.0
3617.5
3622.0
3619.5
3618.0
3621.0
3623.0
3625.5
3606.5
3623.0
3626.0
3622.0
3620.0
3372.0
3372.0
3614.0
3623.0
3615.5
3605.0
3612.0
3615.0
3625.0
3621.0
3624.0
3613.5
3616.0
3620.0
3632.0
3628.0
3630.0
3609.5
3611.5
3889.0
3938.0
3648.0
3605.5
3578.0
3822.0
3664.0
3807.0
3785.5
3598.0
3542.5
3541.0
3541.0
3535.5
3539.0
3541.5
3539.5
3533.0
3605.0
3610.0
3612.0
3635.0
3632.0
3601.0
3613.0
3594.5
3602.5
3568.5
3573.5
3567.5
3572.5
3567.5
3562.5
3347.0
3529.5
3428.0
3528.5
3528.5
3528.0
3499.0
3553.0
3575.5
3522.0
3523.5
3505.0
3504.5
3526.0
3521.0
3491.0
3565.0
3509.0
3505.5
3501.5
3506.0
3498.5
3499.5
3509.5
3502.5
3503.5
3511.5
3507.5
3506.5
3498.5
3504.5
3502.5
3501.5
3505.0
3504.0
3501.5
3497.5